In [13]:
import clip
import torch
from PIL import Image
import numpy as np
import faiss
from sklearn.cluster import KMeans
from pathlib import Path
import os
import shutil
from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, rbd

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Function to get CLIP embeddings for images
def get_clip_embeddings(images, model, preprocess):
    counter=0

    embeddings = []
    for img in images:
        counter = counter+1
        print(counter)
        image = preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = model.encode_image(image)
        embeddings.append(embedding.cpu().numpy())
    return np.vstack(embeddings)

# Function to cluster embeddings
def cluster_embeddings(embeddings, K):
    kmeans = KMeans(n_clusters=K, random_state=0).fit(embeddings)
    return kmeans

# Function to find the nearest cluster center
def find_nearest_cluster_center(embedding, cluster_centers):
    index = faiss.IndexFlatL2(cluster_centers.shape[1])
    index.add(cluster_centers)
    D, I = index.search(embedding, 1)
    return I[0][0]

# Function to match features using LightGlue
def match_features(query_feats, ref_feats, matcher):
    matches = matcher({"image0": query_feats, "image1": ref_feats})
    query_feats, ref_feats, matches = [rbd(x) for x in [query_feats, ref_feats, matches]]
    kpts0, kpts1, matches = query_feats["keypoints"], ref_feats["keypoints"], matches["matches"]
    return len(kpts0[matches[..., 0]])

# Load CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)

# Load LightGlue model
extractor = SuperPoint(max_num_keypoints=1024).eval().to(device)
matcher = LightGlue(features="superpoint").eval().to(device)

# Set paths
dataset_name = "sku100k_v2"
query_imgs_dir = f"./datasets/{dataset_name}/query_images"
class_imgs_dir = f"./datasets/{dataset_name}/reference_images"
output_dir_path = f"./datasets/output/{dataset_name}/top_1"

# Load reference images and query image
reference_image_paths = [os.path.join(class_imgs_dir, path) for path in os.listdir(class_imgs_dir)]
query_image_paths = [os.path.join(query_imgs_dir, path) for path in os.listdir(query_imgs_dir)]

# Get CLIP embeddings for reference images
reference_images = [Image.open(path) for path in reference_image_paths]
ref_embeddings = get_clip_embeddings(reference_images, model, preprocess)

# Cluster reference images into K clusters
K = 10  # Number of clusters
kmeans = cluster_embeddings(ref_embeddings, K)
cluster_centers = kmeans.cluster_centers_
ref_labels = kmeans.labels_
print('ref_labels: ', ref_labels)
# Process each query image
with torch.no_grad():
    for query_img_path in query_image_paths:
        query_image = Image.open(query_img_path)
        query_embedding = get_clip_embeddings([query_image], model, preprocess)
        
        best_cluster_idx = find_nearest_cluster_center(query_embedding, cluster_centers)
        print("best_cluster_idx: ", best_cluster_idx)
        # Extract features for query image
        image0 = load_image(query_img_path)
        query_feats = extractor.extract(image0.to(device))
        
        best_match_score = 0
        best_reference = None
        
        for idx, label in enumerate(ref_labels):
            if label == best_cluster_idx:
                ref_img_path = reference_image_paths[idx]
                image1 = load_image(ref_img_path)
                ref_feats = extractor.extract(image1.to(device))
                
                match_score = match_features(query_feats, ref_feats, matcher)
                
                if match_score > best_match_score:
                    best_match_score = match_score
                    best_reference = ref_img_path

        threshold = 10  # Minimum matching points threshold
        output_dir = os.path.join(output_dir_path, os.path.basename(query_img_path))
        
        if best_match_score >= threshold:
            os.makedirs(output_dir, exist_ok=True)
            shutil.copyfile(query_img_path, os.path.join(output_dir, "query_img_" + os.path.basename(query_img_path)))
            shutil.copyfile(best_reference, os.path.join(output_dir, "matched_" + os.path.basename(best_reference) + "_" + str(best_match_score)))
        else:
            print(f"No match found for {query_img_path}")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


/usr/lib/python3/dist-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


461
462
463


/home/rafay/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


ref_labels:  [4 8 1 1 3 1 1 1 5 1 8 8 5 4 5 5 2 5 5 3 4 7 0 2 6 3 3 4 4 3 3 4 4 4 4 8 6
 3 1 5 2 4 4 0 5 3 3 1 1 4 0 0 3 0 4 0 0 0 3 0 8 1 1 8 2 8 2 4 4 4 1 5 3 3
 8 8 6 0 1 6 6 1 1 9 0 5 6 9 7 8 7 4 3 8 2 3 3 1 7 1 0 3 4 8 8 2 2 8 8 5 4
 3 0 7 4 3 4 8 8 1 3 8 8 2 1 3 7 3 2 1 3 4 4 4 5 8 8 8 3 0 0 0 6 2 2 2 2 2
 1 0 0 0 3 6 6 3 1 3 2 8 3 3 0 0 1 9 5 5 5 5 5 5 5 2 1 1 1 1 3 1 1 3 1 3 4
 3 4 3 8 3 3 3 8 8 4 3 5 5 3 0 4 6 5 0 1 4 7 4 5 5 4 8 3 5 8 3 3 4 3 3 2 1
 2 4 7 7 1 1 1 1 0 3 0 4 4 4 1 1 0 1 3 6 1 2 4 1 5 7 3 8 1 4 4 0 5 5 1 8 4
 0 3 3 4 4 3 7 7 4 4 7 1 7 3 7 3 7 7 3 3 9 9 9 9 9 9 5 9 9 3 0 0 4 3 0 3 0
 3 1 0 4 3 0 3 3 0 4 5 0 5 3 0 3 3 0 0 0 4 0 0 7 0 3 0 0 0 3 4 0 3 1 0 0 0
 0 0 1 0 2 2 5 5 5 1 1 0 0 1 7 0 7 1 1 4 4 3 6 0 1 1 2 7 7 1 3 7 7 7 7 7 7
 1 5 0 8 2 2 2 3 5 5 3 5 5 5 9 5 4 1 6 6 6 6 6 3 3 0 6 4 1 1 8 5 1 1 3 1 1
 3 4 1 4 4 9 9 9 9 9 9 9 9 4 5 5 5 9 9 9 9 1 1 3 3 1 1 7 1 8 2 4 0 0 0 8 1
 1 6 3 9 3 4 4 3 4 4 0 2 4 0 4 4 0 1 1]
1
best_cluster_idx:  4
1
best_cluster_idx:  3
1

In [18]:
import clip
import torch
from PIL import Image
import numpy as np
import faiss
from sklearn.cluster import KMeans
from pathlib import Path
import os
import shutil
import time
from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image, rbd

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get CLIP embeddings for images
def get_clip_embeddings(images, model, preprocess):
    embeddings = []
    for img in images:
        image = preprocess(img).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = model.encode_image(image)
        embeddings.append(embedding.cpu().numpy())
    return np.vstack(embeddings)

# Function to cluster embeddings
def cluster_embeddings(embeddings, K):
    kmeans = KMeans(n_clusters=K, random_state=0).fit(embeddings)
    return kmeans

# Function to find the nearest cluster center
def find_nearest_cluster_center(embedding, cluster_centers):
    index = faiss.IndexFlatL2(cluster_centers.shape[1])
    index.add(cluster_centers)
    D, I = index.search(embedding, 1)
    return I[0][0]

# Function to match features using LightGlue
def match_features(query_feats, ref_feats, matcher):
    matches = matcher({"image0": query_feats, "image1": ref_feats})
    query_feats, ref_feats, matches = [rbd(x) for x in [query_feats, ref_feats, matches]]
    kpts0, kpts1, matches = query_feats["keypoints"], ref_feats["keypoints"], matches["matches"]
    return len(kpts0[matches[..., 0]])

# Load CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)

# Load LightGlue model
extractor = SuperPoint(max_num_keypoints=1024).eval().to(device)
matcher = LightGlue(features="superpoint").eval().to(device)

# Set paths
dataset_name = "sku100k_v2"
query_imgs_dir = f"./datasets/{dataset_name}/query_images"
class_imgs_dir = f"./datasets/{dataset_name}/reference_images"
output_dir_path = f"./datasets/output/{dataset_name}/top_1eval"

# Load reference images and query image
reference_image_paths = [os.path.join(class_imgs_dir, path) for path in os.listdir(class_imgs_dir)]
query_image_paths = [os.path.join(query_imgs_dir, path) for path in os.listdir(query_imgs_dir)]

# Get CLIP embeddings for reference images
reference_images = [Image.open(path) for path in reference_image_paths]
print('generating embeddings')
ref_embeddings = get_clip_embeddings(reference_images, model, preprocess)
print('embeddings done')

# Cluster reference images into K clusters
K = 10  # Number of clusters
kmeans = cluster_embeddings(ref_embeddings, K)
cluster_centers = kmeans.cluster_centers_
ref_labels = kmeans.labels_

# Timing code
start_time = time.time()
n_query_imgs = 0

with torch.no_grad():
    print('Starting...')
    
    for query_index, query_img_path in enumerate(query_image_paths):
        print("Query index: ", query_index)
        query_image = Image.open(query_img_path)
        query_embedding = get_clip_embeddings([query_image], model, preprocess)
        
        best_cluster_idx = find_nearest_cluster_center(query_embedding, cluster_centers)
        
        # Extract features for query image
        image0 = load_image(query_img_path)
        query_feats = extractor.extract(image0.to(device))
        
        best_match_score = 0
        best_reference = None
        filenames = {}
        cluster_start_time = time.time()
        
        for idx, label in enumerate(ref_labels):
            if label == best_cluster_idx:
                ref_img_path = reference_image_paths[idx]
                image1 = load_image(ref_img_path)
                ref_feats = extractor.extract(image1.to(device))
                
                match_score = match_features(query_feats, ref_feats, matcher)
                
                if match_score > best_match_score:
                    best_match_score = match_score
                    best_reference = ref_img_path
                filenames[ref_img_path] = match_score

        cluster_end_time = time.time()
        print(f"Each query image takes: {cluster_end_time - cluster_start_time:.2f} seconds")
        n_query_imgs += 1
        filenames = {k: v for k, v in sorted(filenames.items(), reverse=True, key=lambda item: item[1])}

        threshold = 1  # Top 1 match
        output_dir = os.path.join(output_dir_path, os.path.basename(query_img_path))
        
        if best_match_score >= threshold:
            os.makedirs(output_dir, exist_ok=True)
            shutil.copyfile(query_img_path, os.path.join(output_dir, "query_img_" + os.path.basename(query_img_path)))
            for index, (reference_img_path, match_score) in enumerate(filenames.items()):
                if index >= threshold:
                    break
                shutil.copyfile(reference_img_path, os.path.join(output_dir, os.path.basename(reference_img_path) + "_" + str(match_score)))
        else:
            print(f"No match found for {query_img_path}")

end_time = time.time()
print(f"Total time: {end_time - start_time:.2f} seconds")
print("Query images processed: ", n_query_imgs)
print("Total comparisons: ", n_query_imgs * len(ref_labels))


generating embeddings


/usr/lib/python3/dist-packages/PIL/Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


embeddings done


/home/rafay/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Starting...
Query index:  0


KeyboardInterrupt: 